-  PARTICIPANTES_2024 me dá se a condição financeira do candidato/escola, porém nao tem chave com as escolas

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_seq_items', None)

In [ ]:
df_raw = pd.read_csv("../dados/inputs/RESULTADOS_2024.csv", sep=";", encoding="latin1")

df_raw.info()  # 4.3M linhas

In [ ]:
# Remover linhas que nao tenham codigo escola
df = df_raw.dropna(subset="CO_ESCOLA")  # 1.5M linhas

# Remove quem faltou qualquer dia
cols = ["TP_PRESENCA_CN", "TP_PRESENCA_CH", "TP_PRESENCA_LC", "TP_PRESENCA_MT"]
df = df[df[cols].eq(1).all(axis=1)]  # 1.19M linhas

# Remove escolas com menos de 5 provas
df = df.groupby('CO_ESCOLA').filter(lambda g: len(g) >= 5)  # 1.18M linhas

df = df[["CO_ESCOLA", "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_REDACAO"]]

df.info()


In [ ]:
df_escolas = df.groupby("CO_ESCOLA", as_index=False).agg("mean")  # 26_302 escolas

df_escolas["nota_objetiva"] = df_escolas[["NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT"]].mean(axis=1)

df_escolas["cod_escola"] = df_escolas["CO_ESCOLA"].astype(int).astype(str)  # estava com .0 no final


df_escolas.info()

In [ ]:
df_final = df_escolas[["cod_escola", "nota_objetiva", "NU_NOTA_REDACAO"]]

df_final = df_final.rename(columns={"NU_NOTA_REDACAO": "nota_redacao"})

df_final.to_csv("../dados/temporarios/escolas_enem.csv", index=False)

In [ ]:
df_final.head()